<a href="https://colab.research.google.com/github/Saketh-11653883/UNT-SAKETH_INFO5731/blob/main/INFO5731_Assignment_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
import requests
import csv
from bs4 import BeautifulSoup

def get_total_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    total_reviews_str = soup.find('div', {'class': 'header'}).get_text().split()[0]
    total_reviews = int(total_reviews_str.replace(',', ''))
    return total_reviews

def scrape_reviews(url, csv_filename):
    total_reviews = get_total_reviews(url)

    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Review'])

        for page in range(1, min(1001, total_reviews // 10 + 2)):
            page_url = f'{url}&start={10*(page-1)}'
            page_response = requests.get(page_url)
            page_soup = BeautifulSoup(page_response.content, 'html.parser')
            reviews = page_soup.find_all('div', {'class': 'text show-more__control'})

            for review in reviews:
                review_text = review.get_text().strip()
                csv_writer.writerow([review_text])

if __name__ == "__main__":
    imdb_url = 'https://www.imdb.com/title/tt10954600/reviews?ref_=tt_urv'
    csv_filename = 'Antman_reviews.csv'
    scrape_reviews(imdb_url, csv_filename)


In [2]:
# Import the necessary library
import pandas as pd

# Load the CSV file data into a DataFrame
df = pd.read_csv('Antman_reviews.csv')

# Display the first 100 rows of the DataFrame
print(df.head(100))

                                               Review
0   As a huge fan of the first one, and almost as ...
1   After most entries in Phase 4 passed by withou...
2   Well it's happened. The MCU has run out of gas...
3   Well, I'll start off by saying that this wasn'...
4   I enjoyed watching Quantumania. It's a mostly ...
..                                                ...
95  This film is unspeakably bad.It's actually wor...
96  A fun one!I had a terrific time watching 'Ant-...
97  As many others have pointed out we are now far...
98  The MCU in its current state is an absolute me...
99  So, I'm not going to say this is a great Marve...

[100 rows x 1 columns]


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [3]:
pip install textblob

In [4]:
# Write your code here
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import re
import pandas as pd

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Noise Removal
df['Reviews after Noise Removal'] = df['Review'].str.replace('[^\w\s]', '')
df['Reviews after Noise Removal'] = df['Reviews after Noise Removal'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x))
df.head()

<ipython-input-5-0238c75436c1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Reviews after Noise Removal'] = df['Review'].str.replace('[^\w\s]', '')


,Review,Reviews after Noise Removal
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...


In [6]:
# Remove Digits
df['After digits removal'] = df['Reviews after Noise Removal'].apply(lambda y: ''.join([i for i in y if not i.isdigit()]))
df.head()

,Review,Reviews after Noise Removal,After digits removal
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...,As a huge fan of the first one and almost as b...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...,After most entries in Phase passed by without...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...,Well its happened The MCU has run out of gas T...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...,Well Ill start off by saying that this wasnt a...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its a mostly so...


In [7]:
# Stopwords Removal
s = stopwords.words('english')
df['Stopwords Removal'] = df['After digits removal'].apply(lambda x: " ".join(x for x in x.split() if x not in s))
df.head()

,Review,Reviews after Noise Removal,After digits removal,Stopwords Removal
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...,As a huge fan of the first one and almost as b...,As huge fan first one almost big fan second on...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...,After most entries in Phase passed by without...,After entries Phase passed without much set ne...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...,Well its happened The MCU has run out of gas T...,Well happened The MCU run gas The last MCU fil...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...,Well Ill start off by saying that this wasnt a...,Well Ill start saying wasnt bad movie It wasnt...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its mostly soli...


In [8]:
# Convert to Lower Case
df['Lower Case'] = df['Stopwords Removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df.head()

,Review,Reviews after Noise Removal,After digits removal,Stopwords Removal,Lower Case
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...,As a huge fan of the first one and almost as b...,As huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...,After most entries in Phase passed by without...,After entries Phase passed without much set ne...,after entries phase passed without much set ne...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...,Well its happened The MCU has run out of gas T...,Well happened The MCU run gas The last MCU fil...,well happened the mcu run gas the last mcu fil...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...,Well Ill start off by saying that this wasnt a...,Well Ill start saying wasnt bad movie It wasnt...,well ill start saying wasnt bad movie it wasnt...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its mostly soli...,i enjoyed watching quantumania its mostly soli...


In [9]:
# Stemming
stemmer = PorterStemmer()
df['After Stemming'] = df['Lower Case'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))
df.head()

,Review,Reviews after Noise Removal,After digits removal,Stopwords Removal,Lower Case,After Stemming
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...,As a huge fan of the first one and almost as b...,As huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...,After most entries in Phase passed by without...,After entries Phase passed without much set ne...,after entries phase passed without much set ne...,after entri phase pass without much set next b...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...,Well its happened The MCU has run out of gas T...,Well happened The MCU run gas The last MCU fil...,well happened the mcu run gas the last mcu fil...,well happen the mcu run ga the last mcu film l...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...,Well Ill start off by saying that this wasnt a...,Well Ill start saying wasnt bad movie It wasnt...,well ill start saying wasnt bad movie it wasnt...,well ill start say wasnt bad movi it wasnt gre...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its mostly soli...,i enjoyed watching quantumania its mostly soli...,i enjoy watch quantumania it mostli solid fair...


In [10]:
import nltk
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
# Lemmatization
df['After Lemmatization'] = df['After Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df.head()

,Review,Reviews after Noise Removal,After digits removal,Stopwords Removal,Lower Case,After Stemming,After Lemmatization
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...,As a huge fan of the first one and almost as b...,As huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...,a huge fan first one almost big fan second one...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...,After most entries in Phase passed by without...,After entries Phase passed without much set ne...,after entries phase passed without much set ne...,after entri phase pass without much set next b...,after entri phase pas without much set next bi...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...,Well its happened The MCU has run out of gas T...,Well happened The MCU run gas The last MCU fil...,well happened the mcu run gas the last mcu fil...,well happen the mcu run ga the last mcu film l...,well happen the mcu run ga the last mcu film l...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...,Well Ill start off by saying that this wasnt a...,Well Ill start saying wasnt bad movie It wasnt...,well ill start saying wasnt bad movie it wasnt...,well ill start say wasnt bad movi it wasnt gre...,well ill start say wasnt bad movi it wasnt gre...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its mostly soli...,i enjoyed watching quantumania its mostly soli...,i enjoy watch quantumania it mostli solid fair...,i enjoy watch quantumania it mostli solid fair...


In [12]:
df['cleaned_text']= df['After Lemmatization']

In [13]:
# Save the cleaned data to a new column and CSV file
df.to_csv('Antman_reviews_cleaned.csv', index=False)

# Display the first few rows of the DataFrame
df.head()

,Review,Reviews after Noise Removal,After digits removal,Stopwords Removal,Lower Case,After Stemming,After Lemmatization,cleaned_text
0,"As a huge fan of the first one, and almost as ...",As a huge fan of the first one and almost as b...,As a huge fan of the first one and almost as b...,As huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...,as huge fan first one almost big fan second on...,a huge fan first one almost big fan second one...,a huge fan first one almost big fan second one...
1,After most entries in Phase 4 passed by withou...,After most entries in Phase 4 passed by withou...,After most entries in Phase passed by without...,After entries Phase passed without much set ne...,after entries phase passed without much set ne...,after entri phase pass without much set next b...,after entri phase pas without much set next bi...,after entri phase pas without much set next bi...
2,Well it's happened. The MCU has run out of gas...,Well its happened The MCU has run out of gas T...,Well its happened The MCU has run out of gas T...,Well happened The MCU run gas The last MCU fil...,well happened the mcu run gas the last mcu fil...,well happen the mcu run ga the last mcu film l...,well happen the mcu run ga the last mcu film l...,well happen the mcu run ga the last mcu film l...
3,"Well, I'll start off by saying that this wasn'...",Well Ill start off by saying that this wasnt a...,Well Ill start off by saying that this wasnt a...,Well Ill start saying wasnt bad movie It wasnt...,well ill start saying wasnt bad movie it wasnt...,well ill start say wasnt bad movi it wasnt gre...,well ill start say wasnt bad movi it wasnt gre...,well ill start say wasnt bad movi it wasnt gre...
4,I enjoyed watching Quantumania. It's a mostly ...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its a mostly so...,I enjoyed watching Quantumania Its mostly soli...,i enjoyed watching quantumania its mostly soli...,i enjoy watch quantumania it mostli solid fair...,i enjoy watch quantumania it mostli solid fair...,i enjoy watch quantumania it mostli solid fair...


In [14]:
import pandas as pd

In [15]:
df.columns

Index(['Review', 'Reviews after Noise Removal', 'After digits removal',
       'Stopwords Removal', 'Lower Case', 'After Stemming',
       'After Lemmatization', 'cleaned_text'],
      dtype='object')

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [16]:
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')

In [17]:
!pip install nltk
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [22]:
import spacy
import nltk
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tree import Tree
from collections import Counter
import pandas as pd

# Load the spaCy English model for dependency parsing and named entity recognition.
nlp = spacy.load("en_core_web_sm")

# Function to print constituency parsing tree using NLTK.
def print_constituency_tree(text):
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged = pos_tag(words)
        chunked = ne_chunk(tagged)
        for subtree in chunked:
            if type(subtree) == Tree:
                print(subtree.label(), " ".join(word for word, pos in subtree.leaves()))
            else:
                print(subtree[0], subtree[1])

# Function to print dependency parsing tree using spaCy.
def print_dependency_tree(text):
    doc = nlp(text)
    for token in doc:
        print(token.text, token.dep_, token.head.text, [child for child in token.children])

# Function to extract named entities and count their occurrences.
def extract_named_entities(text):
    doc = nlp(text)
    entity_counter = Counter()
    for ent in doc.ents:
        entity_counter[ent.label_] += 1
    return entity_counter

# Read only the first 100 rows from the CSV file to avoid run time issues
df = pd.read_csv('Antman_reviews_cleaned.csv', nrows=100)

# Combine the 'cleaned_text' column values into a single text
text = ' '.join(df['cleaned_text'].astype(str))


# (1) Parts of Speech (POS) Tagging
tokens = word_tokenize(text)
pos_tags = pos_tag(tokens)

# Count the number of nouns, verbs, adjectives, and adverbs in the text.
noun_count = len([word for word, pos in pos_tags if pos.startswith('N')])
verb_count = len([word for word, pos in pos_tags if pos.startswith('V')])
adj_count = len([word for word, pos in pos_tags if pos.startswith('J')])
adv_count = len([word for word, pos in pos_tags if pos.startswith('R')])

print(f"Total Nouns: {noun_count}")
print(f"Total Verbs: {verb_count}")
print(f"Total Adjectives: {adj_count}")
print(f"Total Adverbs: {adv_count}")

# (2) Constituency Parsing and Dependency Parsing
print("Constituency Parsing Trees:")
print_constituency_tree(text)
print("\nDependency Parsing Tree:")
print_dependency_tree(text)

# (3) Named Entity Recognition
entity_counter = extract_named_entities(text)
print("Named Entities:")
for entity, count in entity_counter.items():
    print(f"{entity}: {count}")


Streaming output truncated to the last 5000 lines.
excus ccomp repeat [charact]
introduc npadvmod shangchi []
new amod charact []
charact compound definit [new]
definit nsubj go [charact]
go ccomp shangchi [definit, stori]
stori dobj go [make]
that nsubj make []
make relcl stori [that, charact, import, feel]
paul nsubj charact []
rudd compound charact []
charact ccomp make [paul, rudd]
le nmod import []
import ccomp make [le]
you nsubj feel []
do aux feel []
nt neg feel []
feel ccomp make [you, do, nt, stori]
stori dobj feel []
he nsubj shangchi []
and cc shangchi []
stori conj shangchi [good]
even advmod good []
good advmod stori [even]
you nsubj feel []
feel ccomp shangchi [you, lack]
lack xcomp feel [know]
great amod movi []
charact compound previou []
previou compound movi [charact]
movi nsubj know [great, previou, set]
they nsubj set []
set relcl movi [they, stori]
tone compound stori []
stori dobj set [tone]
do aux know []
nt neg know []
even advmod know []
know parataxis lack [m

Constituency parsing involves breaking down a sentence into constituents (phrases), and the hierarchical structure reflects how these constituents are nested within one another. In the example "the first antman movie," "antman movie" is a noun phrase, while "the first" is an adjective. Constituency parsing represents these relationships as a tree structure.Dependency parsing focuses on the relationships between words in a phrase, where one word is dependent on another. For example, "best mediocre" demonstrates a reliance connection, with "mediocre" relying on "best." The dependencies form a tree structure that depicts the syntactic relationships between words in the phrase.

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

There are many different NLP activities covered in this assignmengt. It also offers a great chance to learn about and put important natural language processing concepts into practice.comparing to the last assignment i felt this assignment little bit easy and time given for the task is enough.